# 第3课：上下文管理器

## 学习目标
- 理解上下文管理器的概念
- 掌握 with 语句的使用
- 学会创建自定义上下文管理器
- 了解 contextlib 模块

## 1. 什么是上下文管理器？

上下文管理器用于管理资源的获取和释放，确保资源被正确清理。

In [ ]:
# 不使用上下文管理器
f = open("test.txt", "w")
try:
    f.write("Hello")
finally:
    f.close()

# 使用上下文管理器
with open("test.txt", "w") as f:
    f.write("Hello")
# 文件自动关闭

print("文件操作完成")

## 2. 自定义上下文管理器（类方式）

In [ ]:
# 实现 __enter__ 和 __exit__ 方法
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        print(f"打开文件: {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"关闭文件: {self.filename}")
        if self.file:
            self.file.close()
        # 返回 False 表示不抑制异常
        return False

with FileManager("test.txt", "w") as f:
    f.write("Hello, Context Manager!")

In [ ]:
# 处理异常的上下文管理器
class ErrorHandler:
    def __enter__(self):
        print("进入上下文")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            print(f"捕获异常: {exc_type.__name__}: {exc_val}")
            return True  # 抑制异常
        print("正常退出")
        return False

with ErrorHandler():
    print("执行代码")
    raise ValueError("测试错误")
    print("这行不会执行")

print("程序继续运行")

In [ ]:
# 计时上下文管理器
import time

class Timer:
    def __init__(self, name=""):
        self.name = name
    
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, *args):
        self.elapsed = time.time() - self.start
        print(f"{self.name} 耗时: {self.elapsed:.4f}秒")
        return False

with Timer("计算"):
    total = sum(range(1000000))
    print(f"结果: {total}")

## 3. 使用 contextlib

In [ ]:
from contextlib import contextmanager

# 使用生成器创建上下文管理器
@contextmanager
def file_manager(filename, mode):
    print(f"打开文件: {filename}")
    f = open(filename, mode)
    try:
        yield f  # yield 之前是 __enter__，yield 的值是返回值
    finally:
        f.close()  # yield 之后是 __exit__
        print(f"关闭文件: {filename}")

with file_manager("test.txt", "w") as f:
    f.write("使用 contextmanager!")

In [ ]:
# 简单的计时器
@contextmanager
def timer(name=""):
    start = time.time()
    yield
    elapsed = time.time() - start
    print(f"{name} 耗时: {elapsed:.4f}秒")

with timer("列表创建"):
    lst = [i**2 for i in range(100000)]

In [ ]:
# 临时目录
@contextmanager
def temp_directory():
    import tempfile
    import shutil
    
    dirpath = tempfile.mkdtemp()
    print(f"创建临时目录: {dirpath}")
    try:
        yield dirpath
    finally:
        shutil.rmtree(dirpath)
        print(f"删除临时目录: {dirpath}")

with temp_directory() as tmpdir:
    print(f"使用目录: {tmpdir}")

## 4. contextlib 其他工具

In [ ]:
from contextlib import suppress, redirect_stdout
import io

# suppress - 抑制指定异常
with suppress(FileNotFoundError):
    import os
    os.remove("nonexistent_file.txt")
    
print("继续执行")

In [ ]:
# redirect_stdout - 重定向标准输出
f = io.StringIO()
with redirect_stdout(f):
    print("这会被重定向")
    print("到字符串缓冲区")

output = f.getvalue()
print(f"捕获的输出:\n{output}")

In [ ]:
from contextlib import ExitStack

# ExitStack - 动态管理多个上下文管理器
filenames = ["file1.txt", "file2.txt", "file3.txt"]

# 创建测试文件
for name in filenames:
    with open(name, "w") as f:
        f.write(f"Content of {name}")

# 使用 ExitStack 同时打开多个文件
with ExitStack() as stack:
    files = [stack.enter_context(open(fn)) for fn in filenames]
    for f in files:
        print(f.read())

# 清理
import os
for name in filenames:
    os.remove(name)

## 5. 实用示例

In [ ]:
# 数据库连接上下文管理器
class DatabaseConnection:
    def __init__(self, host, database):
        self.host = host
        self.database = database
        self.connection = None
    
    def __enter__(self):
        print(f"连接到数据库: {self.database}@{self.host}")
        # 模拟数据库连接
        self.connection = {"host": self.host, "db": self.database}
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("关闭数据库连接")
        self.connection = None
        return False
    
    def query(self, sql):
        print(f"执行查询: {sql}")
        return [{"id": 1, "name": "Test"}]

with DatabaseConnection("localhost", "mydb") as db:
    results = db.query("SELECT * FROM users")
    print(f"结果: {results}")

In [ ]:
# 锁上下文管理器
import threading

class Lock:
    def __init__(self, name):
        self.name = name
        self._lock = threading.Lock()
    
    def __enter__(self):
        print(f"获取锁: {self.name}")
        self._lock.acquire()
        return self
    
    def __exit__(self, *args):
        self._lock.release()
        print(f"释放锁: {self.name}")
        return False

lock = Lock("resource_lock")
with lock:
    print("访问共享资源")

## 6. 练习题

### 练习 1：日志上下文管理器
创建一个记录进入和退出日志的上下文管理器

In [ ]:
@contextmanager
def log_context(name):
    # 在这里编写代码
    pass

# 测试
with log_context("数据处理"):
    print("处理数据...")

### 练习 2：临时修改属性
创建一个临时修改对象属性的上下文管理器

In [ ]:
@contextmanager
def temporary_attr(obj, attr, value):
    # 在这里编写代码
    pass

In [ ]:
# 清理
import os
if os.path.exists("test.txt"):
    os.remove("test.txt")

## 7. 本课小结

1. **上下文管理器**：管理资源的获取和释放
2. **with 语句**：自动调用 `__enter__` 和 `__exit__`
3. **类方式**：实现 `__enter__` 和 `__exit__` 方法
4. **@contextmanager**：使用生成器创建上下文管理器
5. **contextlib**：suppress、redirect_stdout、ExitStack 等工具